In [ ]:
#Run on Google Colab to use GPU

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, Add, ReLU, LSTM, Reshape, Concatenate, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l2

from sklearn.metrics import classification_report, confusion_matrix


import pickle
import os
from contextlib import redirect_stdout

In [ ]:
#import MIT data
df_mitbih_test = pd.read_csv('data/original/mitbih_test.csv', header = None)

X_train = pd.read_csv('data/processed/X_train.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
y_train = y_train['187']

X_train_sm = pd.read_csv('data/processed/X_train_sm.csv')
y_train_sm = pd.read_csv('data/processed/y_train_sm.csv')
y_train_sm = y_train_sm['187']

X_val = pd.read_csv('data/processed/X_val.csv')
y_val = pd.read_csv('data/processed/y_val.csv')
y_val = y_val['187']

X_test = df_mitbih_test.drop(187, axis = 1)
y_test = df_mitbih_test[187]


# Reshape the data for 1D CNN
X_train_sm_cnn = np.expand_dims(X_train_sm, axis=2)
X_val_cnn = np.expand_dims(X_val, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2) 

display(X_train_sm_cnn.shape)
display(X_val_cnn.shape)
display(X_test_cnn.shape)

In [ ]:
#Function to plot and save loss and accuracy over epochs from training history
def plot_training_history(history, save_dir, prefix): 
    hist = history.history
    metrics = [m for m in hist.keys() if not m.startswith('val_')]  

    # Create the output folder if it does not exist
    os.makedirs(save_dir, exist_ok=True)

    for m in metrics:
        plt.figure()
        plt.plot(hist[m], label=f'Train {m}')
        if f'val_{m}' in hist:
            plt.plot(hist[f'val_{m}'], label=f'Val {m}')
        plt.xlabel('Epoch')
        plt.ylabel(m)
        plt.title(f'{m} over epochs')
        plt.legend()
        plt.grid(True)

        # Construct filename with prefix and filepath with directory and filename
        filename = f"{prefix}_{m}.png"
        filepath = os.path.join(save_dir, filename)

        # Save figure
        plt.savefig(filepath, format='png', dpi=300, bbox_inches='tight')
        print(f"Saved: {filepath}")
        plt.show()

In [ ]:
#Used CNNs

#CNN1
cnn1 = Sequential([
    Input((187, 1)),
    
    # First Conv Block
    Conv1D(filters=64, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    # Second Conv Block
    Conv1D(filters=64, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    # Third Conv Block
    Conv1D(filters=32, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    # Dense layers
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])


#CNN2, CNN4 Paper 2020
cnn2 = Sequential([
    Input((187, 1)),
    
    # First Conv Block: 5×32
    Conv1D(filters=32, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    # Second Conv Block: 3×64
    Conv1D(filters=64, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    # Third Conv Block: 5×128
    Conv1D(filters=128, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    
    # Fourth Conv Block: 3×256
    Conv1D(filters=256, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    # Dense layers
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])


#CNN3, CNN4 Paper 2020, increased dropout compared to CNN2
cnn3 = Sequential([
    Input((187, 1)),
    
    # First Conv Block
    Conv1D(filters=32, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  
    
    # Second Conv Block
    Conv1D(filters=64, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  
    
    # Third Conv Block
    Conv1D(filters=128, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),  
    
    # Fourth Conv Block
    Conv1D(filters=256, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),  
    
    # Dense layers
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),  
    Dense(5, activation='softmax')
])


#CNN4, CNN4 Paper 2020, changed dropout compared to CNN2 and CNN3
cnn4 = Sequential([
    Input((187, 1)),
    
    # First Conv Block
    Conv1D(filters=32, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),  
    
    # Second Conv Block
    Conv1D(filters=64, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.25), 
    
    # Third Conv Block
    Conv1D(filters=128, kernel_size=5, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  
    
    # Fourth Conv Block
    Conv1D(filters=256, kernel_size=3, padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.35),  
    
    # Dense layers
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),  
    Dense(5, activation='softmax')
])


#CNN5, Paper 2018
# Input
inputs = Input(shape=(187, 1))

# Initial conv layer, 32 filters
x = Conv1D(filters=32, kernel_size=5, padding='same')(inputs)

# Residual Block 1
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 2
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 3 
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 4
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 5 
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Fully connected layers
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)

# output
outputs = Dense(5, activation='softmax')(x)

# create model
cnn5 = Model(inputs=inputs, outputs=outputs)


#CNN6, Paper 2018 added dropout and batch normalization 
inputs = Input(shape=(187, 1))
# Initial conv
x = Conv1D(filters=32, kernel_size=5, padding='same')(inputs)
x = BatchNormalization()(x)  

# Residual Block 1
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)  
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)  
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.2)(x)  

# Residual Block 2
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.2)(x)

# Residual Block 3
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.2)(x)

# Residual Block 4
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.25)(x)

# Residual Block 5
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.25)(x)

# Fully connected layers 
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)  
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)  

# output
outputs = Dense(5, activation='softmax')(x)

# Create model
cnn6 = Model(inputs=inputs, outputs=outputs)


#CNN7, Paper 2018 added batch normalization, without dropout
inputs = Input(shape=(187, 1))

# Initial conv layer
x = Conv1D(filters=32, kernel_size=5, padding='same')(inputs)
x = BatchNormalization()(x) 

# Residual Block 1
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)  
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)  
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 2
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 3
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 4
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Residual Block 5 
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = BatchNormalization()(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)

# Fully Connected Layers 
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)

# Output
outputs = Dense(5, activation='softmax')(x)

# Create model
cnn7 = Model(inputs=inputs, outputs=outputs)



#CNN8, Paper 2018 added dropout, without batch normalization
inputs = Input(shape=(187, 1))

# Initial conv layer
x = Conv1D(filters=32, kernel_size=5, padding='same')(inputs)

# Residual Block 1
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.1)(x) 

# Residual Block 2
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.1)(x)

# Residual Block 3
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.1)(x)

# Residual Block 4
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.1)(x)

# Residual Block 5 
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.1)(x)

# Fully Connected Layers
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)  
x = Dense(32, activation='relu')(x)
x = Dropout(0.1)(x)  

# Output 
outputs = Dense(5, activation='softmax')(x)

# Create model
cnn8 = Model(inputs=inputs, outputs=outputs)



# CNN9 - changed Dropout strategy compared to CNN8
inputs = Input(shape=(187, 1))

#initial conv layer
x = Conv1D(filters=32, kernel_size=5, padding='same')(inputs)

# Residual Block 1
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.1)(x)  

# Residual Block 2
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.15)(x)  

# Residual Block 3
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.2)(x)  

# Residual Block 4
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.25)(x)  

# Residual Block 5 
shortcut = x
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = ReLU()(x)
x = Conv1D(filters=32, kernel_size=5, padding='same')(x)
x = Add()([shortcut, x])
x = ReLU()(x)
x = MaxPooling1D(pool_size=5, strides=2, padding='same')(x)
x = Dropout(0.3)(x)  

# Fully Connected Layers
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.4)(x)  
x = Dense(32, activation='relu')(x)
x = Dropout(0.3)(x)  

#output
outputs = Dense(5, activation='softmax')(x)

cnn9 = Model(inputs=inputs, outputs=outputs)

In [ ]:
#Model summary
cnn1.summary()

cnn2.summary()

cnn3.summary()

cnn4.summary()

cnn5.summary()

cnn6.summary()

cnn7.summary()

cnn8.summary()

cnn9.summary()

In [ ]:
#lr exp dec
# Learning rate with exponential decay
initial_learning_rate = 1e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96)

#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',        # what to monitor 
    patience=20,               # how many epochs with no improvement before stopping
    restore_best_weights=True, 
    min_delta=0.001            #only stop if improvement < 0.001
)

#Compile when lr exp decay
cnn6.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../CNN_output/cnn6_sm_lrexpdec_earlystop_bs512_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',            # metric to monitor
    mode='min',                    # minimize loss
    save_best_only=False,          # save model of every epoch
    verbose=1                      # print message when a model is saved
)

#training
history = cnn6.fit(
    X_train_sm_cnn,
    y_train_sm,
    epochs=200,
    batch_size=512,
    validation_data=(X_val_cnn, y_val),
    callbacks=[checkpoint, early_stop] 
)

In [ ]:
# Learning rate with exponential decay - PAPER 2018
initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.75
)

# Adam optimizer with specified hyperparameters
optimizer = Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999
)

#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',       
    patience=20,              
    restore_best_weights=True, 
    min_delta=0.001  
)

cnn6.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../CNN_output/cnn6_sm_lrrexpdecpaper_earlystop_bs512_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',        
    mode='min',                    
    save_best_only=False,           
    verbose=1                      
)

#training
history = cnn6.fit(
    X_train_sm_cnn,
    y_train_sm,
    epochs=200,
    batch_size=512,
    validation_data=(X_val_cnn, y_val),
    callbacks=[checkpoint, early_stop] 
)

In [ ]:
#lr reduction: lrredpl -> reduce on plateau
lrredpl = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.5,          # Factor by which the learning rate will be reduced
    patience=10,         # Number of epochs with no improvement after which learning rate is reduced
    min_lr=1e-7,         # Minimum learning rate
    verbose=1,           # Print a message when the learning rate is reduced
    min_delta=0.001 
)

#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',       
    patience=20,               
    restore_best_weights=True, 
    min_delta=0.001  
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../CNN_output/cnn6_sm_lrexpdec_earlystop_bs512_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',        
    mode='min',                    
    save_best_only=False,          
    verbose=1                      
)

#training
history = model.fit(
    X_train_sm_cnn,
    y_train_sm,
    epochs=200,
    batch_size=512,
    validation_data=(X_val_cnn, y_val),
    callbacks=[checkpoint, early_stop, lrredpl] 
)

In [ ]:
# plot and save loss and accuracy over epochs from history
plot_training_history(history, save_dir="../CNN_output", prefix="cnn6_sm_lrexpdec_bs512") #change for model

In [ ]:
#Save training history
with open("../CNN_output/cnn1_sm_lr_bs_epoch__valloss_.pkl", "wb") as f: #change for model
    pickle.dump(history.history, f)

    
best_model = load_model('../CNN_output/cnn1_sm_lr_bs_epoch__valloss_.keras') #change for model


#prediction of test data
test_pred = best_model.predict(X_test_cnn)
y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis=1)


#classification report
print(classification_report(y_test_class, y_pred_class, digits=4))


#confusion matrix
print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))


#save results of metrics
with open("../CNN_output/cnn1_sm_lr_bs_epoch__valloss_.txt", "w") as file: #change for model
    
    file.write("\nModel: CNN1\n")#change for model
        
    file.write("\nData augmentation: Smote\n")
    
    file.write("\nConfusion Matrix on test set:\n")
    file.write(str(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions'])))
    
    file.write("\n\nClassification Report on test set:\n")
    file.write(classification_report(y_test_class, y_pred_class, digits=4))